In [2]:
import psycopg2
import pandas as pd
import numpy as np
import glob
import csv

# Project Workflow: Business Analysis

### Data Enginerring
    1. Limited Data enginerring will be performed since it is hard to track the performed sessions prior to installation of the app and database=

### Business Analysis
    1. Determine users with highest unpaid_workout_ratio
    2. Determine users with the most/least skipped workouts for freqeuncy consistency.
    3. Determine an aggregate value score for clients based on the above two metrics.
    


In [3]:
conn=psycopg2.connect(dbname='cabral_fitness',
user='postgres',
password="Maninthemirror1995",
host='34.102.107.137',
port='5432',
sslmode='disable')

cursor=conn.cursor()

#For business analysis
cursor.execute("""SELECT c.name AS NAME, 
workout_number, block_id, b.workouts_per_week, 
e.exercise AS exercise_name, sets, reps, p.weight 
FROM prescriptions p 
LEFT JOIN client c on p.client_id=c.id
LEFT JOIN exercises e on p.exercise_id=e.id
LEFT JOIN blocks b on p.block_id=b.id
WHERE c.name not in  ('Test','Jared')
ORDER BY c.name, block_id, workout_number;
""")
db_prescriptions=cursor.fetchall()



In [4]:
conn.close()

### Adding some columns to my client table that should have been added a while ago. Will be needed for analysis.

In [5]:
conn=psycopg2.connect(dbname='cabral_fitness',
user='postgres',
password="Maninthemirror1995",
host='34.102.107.137',
port='5432',
sslmode='disable')

cursor=conn.cursor()

cursor.execute("""Select * from client;""")
db_client=cursor.fetchall()

cursor.execute("""Select * from actual_to_prescription;""")
db_actual_to_prescription=cursor.fetchall()

conn.close()

In [6]:
db_client=pd.DataFrame(db_client)
db_client=db_client.drop(columns=0)

In [7]:
"""ender VARCHAR(5),
    ADD COLUMN height INTEGER,
    ADD COLUMN weight INTEGER,
    ADD COLUMN goal TEXT,
    ADD COLUMN age INTEGER,
    ADD COLUMN limitations TEXT,
    ADD COLUMN cost_per_session NUMERIC;"""

db_client.columns=['Name', 'gender', 'height', 'weight', 'goal', 'age', 'limitations', 'cost_per_session']
db_client=db_client.set_index('Name')
db_client

,gender,height,weight,goal,age,limitations,cost_per_session
Name,,,,,,,
Daniel,None,None,None,None,None,None,None
Jared,None,None,None,None,None,None,None
Test,None,None,None,None,None,None,None
Masha,None,None,None,None,None,None,None
Mary,None,None,None,None,None,None,None
Shawn,None,None,None,None,None,None,None
Anya,None,None,None,None,None,None,None
Nat,None,None,None,None,None,None,None
Abelev,None,None,None,None,None,None,None


In [8]:
females=['Dassi', 'Sachi', 'Yelena', 'Anya', 'Mary', 'Masha']

db_client['gender'] = db_client.index.to_series().apply(lambda x: 'F' if x in females else 'M')


In [9]:
s_75=['Nat', 'Misha', 'Yelena', 'Sachi', 'Alan', 'Abelev', 'Dassi']

db_client['cost_per_session']=db_client.index.to_series().apply(lambda x: 75 if x in s_75 else 60)

In [10]:
db_client['cost_per_session']['Masha']=70
db_client['cost_per_session']['Mary']=32
db_client['cost_per_session']['Nick']=32

/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_65031/3291243513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_client['cost_per_session']['Masha']=70
/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_65031/3291243513.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_client['cost_per_session']['Mary']=32
/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_65031/3291243513.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [11]:
db_client

,gender,height,weight,goal,age,limitations,cost_per_session
Name,,,,,,,
Daniel,M,None,None,None,None,None,60
Jared,M,None,None,None,None,None,60
Test,M,None,None,None,None,None,60
Masha,F,None,None,None,None,None,70
Mary,F,None,None,None,None,None,32
Shawn,M,None,None,None,None,None,60
Anya,F,None,None,None,None,None,60
Nat,M,None,None,None,None,None,75
Abelev,M,None,None,None,None,None,75


Thats all i need for the business analysis. I'll add the unique user details later

## Part 1: Unpaid Service Ratio Calculation

In [12]:
cost=db_client['cost_per_session'].to_dict()

In [13]:
db_prescriptions=pd.DataFrame(db_prescriptions, columns=['name','workout_number','block_id','workouts_per_week','exercise_name','sets','reps','weight'])
db_prescriptions

,name,workout_number,block_id,workouts_per_week,exercise_name,sets,reps,weight
0,Abelev,0,14,2,Side Planks,3,30,1
1,Abelev,0,14,2,Burpees,3,10,1
2,Abelev,0,14,2,Banded Walks,3,60,1
3,Abelev,0,14,2,Split Squat Jumps,3,10,1
4,Abelev,0,14,2,Banded Squats,3,10,1
...,...,...,...,...,...,...,...,...
2659,Yelena,15,27,4,Mountain Climbers,3,30,1
2660,Yelena,15,27,4,Back Squats,4,5,50
2661,Yelena,15,27,4,RFE Lunges,3,8,1
2662,Yelena,15,27,4,DB OH Press,4,5,10


In [14]:
db_prescriptions

,name,workout_number,block_id,workouts_per_week,exercise_name,sets,reps,weight
0,Abelev,0,14,2,Side Planks,3,30,1
1,Abelev,0,14,2,Burpees,3,10,1
2,Abelev,0,14,2,Banded Walks,3,60,1
3,Abelev,0,14,2,Split Squat Jumps,3,10,1
4,Abelev,0,14,2,Banded Squats,3,10,1
...,...,...,...,...,...,...,...,...
2659,Yelena,15,27,4,Mountain Climbers,3,30,1
2660,Yelena,15,27,4,Back Squats,4,5,50
2661,Yelena,15,27,4,RFE Lunges,3,8,1
2662,Yelena,15,27,4,DB OH Press,4,5,10


In [15]:
#Groups by name and block and finds the max workout_number which corresponds to teh total number of workouts in that block
result=db_prescriptions.groupby(['name', 'block_id']).aggregate({'workout_number':'max',
                                                                 'workouts_per_week':'max'})
result

workout_number  workouts_per_week
name    block_id                                   
Abelev  14                     7                  2
        32                    11                  3
Alan    19                     7                  2
        39                     7                  2
Anya    6                      5                  2
        30                     7                  2
Arne    18                     7                  2
        41                     7                  2
Daniel  13                     5                  2
        33                     7                  2
Dassi   8                      7                  2
        40                     7                  2
Dimitri 10                     7                  2
        34                     7                  2
Dino    21                     7                  2
        37                     7                  2
Igor    7                      7                  2
        46                     7                  2
Mary    17                    11                  3
        43                    11                  3
Masha   2                      7                  2
        28                     7                  2
Misha   1                     15                  4
        36                    15                  4
Nat     5                     11                  3
        31                    11                  3
Nick    16                    11                  3
        42                    11                  3
Sachi   9                      7                  2
        29                     7                  2
Shawn   15                    11                  3
        35                    11                  3
Yelena  4                     15                  4
        27                    15                  4

In [16]:
result.sort_values(by='workout_number', ascending=False)

workout_number  workouts_per_week
name    block_id                                   
Yelena  27                    15                  4
        4                     15                  4
Misha   1                     15                  4
        36                    15                  4
Abelev  32                    11                  3
Mary    43                    11                  3
Nat     5                     11                  3
        31                    11                  3
Nick    16                    11                  3
        42                    11                  3
Shawn   15                    11                  3
        35                    11                  3
Mary    17                    11                  3
Sachi   9                      7                  2
        29                     7                  2
Masha   28                     7                  2
        2                      7                  2
Abelev  14                     7                  2
Igor    7                      7                  2
Dino    37                     7                  2
        21                     7                  2
Dimitri 34                     7                  2
        10                     7                  2
Dassi   40                     7                  2
        8                      7                  2
Daniel  33                     7                  2
Arne    41                     7                  2
        18                     7                  2
Anya    30                     7                  2
Alan    39                     7                  2
        19                     7                  2
Igor    46                     7                  2
Daniel  13                     5                  2
Anya    6                      5                  2

In [17]:
result['cost_per_session']=result.index.get_level_values(0).map(cost)
result

workout_number  workouts_per_week  cost_per_session
name    block_id                                                     
Abelev  14                     7                  2                75
        32                    11                  3                75
Alan    19                     7                  2                75
        39                     7                  2                75
Anya    6                      5                  2                60
        30                     7                  2                60
Arne    18                     7                  2                60
        41                     7                  2                60
Daniel  13                     5                  2                60
        33                     7                  2                60
Dassi   8                      7                  2                75
        40                     7                  2                75
Dimitri 10                     7                  2                60
        34                     7                  2                60
Dino    21                     7                  2                60
        37                     7                  2                60
Igor    7                      7                  2                60
        46                     7                  2                60
Mary    17                    11                  3                32
        43                    11                  3                32
Masha   2                      7                  2                70
        28                     7                  2                70
Misha   1                     15                  4                75
        36                    15                  4                75
Nat     5                     11                  3                75
        31                    11                  3                75
Nick    16                    11                  3                32
        42                    11                  3                32
Sachi   9                      7                  2                75
        29                     7                  2                75
Shawn   15                    11                  3                60
        35                    11                  3                60
Yelena  4                     15                  4                75
        27                    15                  4                75

In [18]:
result['workouts_per_week']=2
result['workouts_per_week']

name     block_id
Abelev   14          2
         32          2
Alan     19          2
         39          2
Anya     6           2
         30          2
Arne     18          2
         41          2
Daniel   13          2
         33          2
Dassi    8           2
         40          2
Dimitri  10          2
         34          2
Dino     21          2
         37          2
Igor     7           2
         46          2
Mary     17          2
         43          2
Masha    2           2
         28          2
Misha    1           2
         36          2
Nat      5           2
         31          2
Nick     16          2
         42          2
Sachi    9           2
         29          2
Shawn    15          2
         35          2
Yelena   4           2
         27          2
Name: workouts_per_week, dtype: int64

In [19]:
result['workouts_per_week']['Yelena']=0.5
result['workouts_per_week']['Shawn']=3
result['workouts_per_week']['Ilya']=1
result['workouts_per_week']['Daniel']=1


/var/folders/wv/c3g7vgxd10n0mdl2_flcjj480000gn/T/ipykernel_65031/1652406920.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['workouts_per_week']['Daniel']=1


In [20]:
result

workout_number  workouts_per_week  cost_per_session
name    block_id                                                     
Abelev  14                     7                2.0                75
        32                    11                2.0                75
Alan    19                     7                2.0                75
        39                     7                2.0                75
Anya    6                      5                2.0                60
        30                     7                2.0                60
Arne    18                     7                2.0                60
        41                     7                2.0                60
Daniel  13                     5                1.0                60
        33                     7                1.0                60
Dassi   8                      7                2.0                75
        40                     7                2.0                75
Dimitri 10                     7                2.0                60
        34                     7                2.0                60
Dino    21                     7                2.0                60
        37                     7                2.0                60
Igor    7                      7                2.0                60
        46                     7                2.0                60
Mary    17                    11                2.0                32
        43                    11                2.0                32
Masha   2                      7                2.0                70
        28                     7                2.0                70
Misha   1                     15                2.0                75
        36                    15                2.0                75
Nat     5                     11                2.0                75
        31                    11                2.0                75
Nick    16                    11                2.0                32
        42                    11                2.0                32
Sachi   9                      7                2.0                75
        29                     7                2.0                75
Shawn   15                    11                3.0                60
        35                    11                3.0                60
Yelena  4                     15                0.5                75
        27                    15                0.5                75

In [21]:
result.rename(mapper={'workout_number':'total_workouts'}, axis=1, inplace=True)
result

total_workouts  workouts_per_week  cost_per_session
name    block_id                                                     
Abelev  14                     7                2.0                75
        32                    11                2.0                75
Alan    19                     7                2.0                75
        39                     7                2.0                75
Anya    6                      5                2.0                60
        30                     7                2.0                60
Arne    18                     7                2.0                60
        41                     7                2.0                60
Daniel  13                     5                1.0                60
        33                     7                1.0                60
Dassi   8                      7                2.0                75
        40                     7                2.0                75
Dimitri 10                     7                2.0                60
        34                     7                2.0                60
Dino    21                     7                2.0                60
        37                     7                2.0                60
Igor    7                      7                2.0                60
        46                     7                2.0                60
Mary    17                    11                2.0                32
        43                    11                2.0                32
Masha   2                      7                2.0                70
        28                     7                2.0                70
Misha   1                     15                2.0                75
        36                    15                2.0                75
Nat     5                     11                2.0                75
        31                    11                2.0                75
Nick    16                    11                2.0                32
        42                    11                2.0                32
Sachi   9                      7                2.0                75
        29                     7                2.0                75
Shawn   15                    11                3.0                60
        35                    11                3.0                60
Yelena  4                     15                0.5                75
        27                    15                0.5                75

In [22]:
result['total_workouts']=result['total_workouts']+1

In [23]:
result

total_workouts  workouts_per_week  cost_per_session
name    block_id                                                     
Abelev  14                     8                2.0                75
        32                    12                2.0                75
Alan    19                     8                2.0                75
        39                     8                2.0                75
Anya    6                      6                2.0                60
        30                     8                2.0                60
Arne    18                     8                2.0                60
        41                     8                2.0                60
Daniel  13                     6                1.0                60
        33                     8                1.0                60
Dassi   8                      8                2.0                75
        40                     8                2.0                75
Dimitri 10                     8                2.0                60
        34                     8                2.0                60
Dino    21                     8                2.0                60
        37                     8                2.0                60
Igor    7                      8                2.0                60
        46                     8                2.0                60
Mary    17                    12                2.0                32
        43                    12                2.0                32
Masha   2                      8                2.0                70
        28                     8                2.0                70
Misha   1                     16                2.0                75
        36                    16                2.0                75
Nat     5                     12                2.0                75
        31                    12                2.0                75
Nick    16                    12                2.0                32
        42                    12                2.0                32
Sachi   9                      8                2.0                75
        29                     8                2.0                75
Shawn   15                    12                3.0                60
        35                    12                3.0                60
Yelena  4                     16                0.5                75
        27                    16                0.5                75

In [24]:
result['weeks']=4

In [25]:
result

total_workouts  workouts_per_week  cost_per_session  weeks
name    block_id                                                            
Abelev  14                     8                2.0                75      4
        32                    12                2.0                75      4
Alan    19                     8                2.0                75      4
        39                     8                2.0                75      4
Anya    6                      6                2.0                60      4
        30                     8                2.0                60      4
Arne    18                     8                2.0                60      4
        41                     8                2.0                60      4
Daniel  13                     6                1.0                60      4
        33                     8                1.0                60      4
Dassi   8                      8                2.0                75      4
        40                     8                2.0                75      4
Dimitri 10                     8                2.0                60      4
        34                     8                2.0                60      4
Dino    21                     8                2.0                60      4
        37                     8                2.0                60      4
Igor    7                      8                2.0                60      4
        46                     8                2.0                60      4
Mary    17                    12                2.0                32      4
        43                    12                2.0                32      4
Masha   2                      8                2.0                70      4
        28                     8                2.0                70      4
Misha   1                     16                2.0                75      4
        36                    16                2.0                75      4
Nat     5                     12                2.0                75      4
        31                    12                2.0                75      4
Nick    16                    12                2.0                32      4
        42                    12                2.0                32      4
Sachi   9                      8                2.0                75      4
        29                     8                2.0                75      4
Shawn   15                    12                3.0                60      4
        35                    12                3.0                60      4
Yelena  4                     16                0.5                75      4
        27                    16                0.5                75      4

In [26]:
result['cost_of_conducted_workouts']=result['cost_per_session']*result['weeks']*result['workouts_per_week']
result['written_workouts_cost'] = result['total_workouts'] * result['cost_per_session']
result['ROI']=(result['cost_of_conducted_workouts']-result['written_workouts_cost'])/result['written_workouts_cost']
result


total_workouts  workouts_per_week  cost_per_session  weeks  \
name    block_id                                                               
Abelev  14                     8                2.0                75      4   
        32                    12                2.0                75      4   
Alan    19                     8                2.0                75      4   
        39                     8                2.0                75      4   
Anya    6                      6                2.0                60      4   
        30                     8                2.0                60      4   
Arne    18                     8                2.0                60      4   
        41                     8                2.0                60      4   
Daniel  13                     6                1.0                60      4   
        33                     8                1.0                60      4   
Dassi   8                      8                2.0                75      4   
        40                     8                2.0                75      4   
Dimitri 10                     8                2.0                60      4   
        34                     8                2.0                60      4   
Dino    21                     8                2.0                60      4   
        37                     8                2.0                60      4   
Igor    7                      8                2.0                60      4   
        46                     8                2.0                60      4   
Mary    17                    12                2.0                32      4   
        43                    12                2.0                32      4   
Masha   2                      8                2.0                70      4   
        28                     8                2.0                70      4   
Misha   1                     16                2.0                75      4   
        36                    16                2.0                75      4   
Nat     5                     12                2.0                75      4   
        31                    12                2.0                75      4   
Nick    16                    12                2.0                32      4   
        42                    12                2.0                32      4   
Sachi   9                      8                2.0                75      4   
        29                     8                2.0                75      4   
Shawn   15                    12                3.0                60      4   
        35                    12                3.0                60      4   
Yelena  4                     16                0.5                75      4   
        27                    16                0.5                75      4   

                  cost_of_conducted_workouts  written_workouts_cost       ROI  
name    block_id                                                               
Abelev  14                             600.0                    600  0.000000  
        32                             600.0                    900 -0.333333  
Alan    19                             600.0                    600  0.000000  
        39                             600.0                    600  0.000000  
Anya    6                              480.0                    360  0.333333  
        30                             480.0                    480  0.000000  
Arne    18                             480.0                    480  0.000000  
        41                             480.0                    480  0.000000  
Daniel  13                             240.0                    360 -0.333333  
        33                             240.0                    480 -0.500000  
Dassi   8                              600.0                    600  0.000000  
        40                             600.0                    600  0.000000  
Dimitri 10       

In [27]:
result.rename(mapper={'ROI': 'unpaid_service_ratio'}, axis=1, inplace=True)
result

total_workouts  workouts_per_week  cost_per_session  weeks  \
name    block_id                                                               
Abelev  14                     8                2.0                75      4   
        32                    12                2.0                75      4   
Alan    19                     8                2.0                75      4   
        39                     8                2.0                75      4   
Anya    6                      6                2.0                60      4   
        30                     8                2.0                60      4   
Arne    18                     8                2.0                60      4   
        41                     8                2.0                60      4   
Daniel  13                     6                1.0                60      4   
        33                     8                1.0                60      4   
Dassi   8                      8                2.0                75      4   
        40                     8                2.0                75      4   
Dimitri 10                     8                2.0                60      4   
        34                     8                2.0                60      4   
Dino    21                     8                2.0                60      4   
        37                     8                2.0                60      4   
Igor    7                      8                2.0                60      4   
        46                     8                2.0                60      4   
Mary    17                    12                2.0                32      4   
        43                    12                2.0                32      4   
Masha   2                      8                2.0                70      4   
        28                     8                2.0                70      4   
Misha   1                     16                2.0                75      4   
        36                    16                2.0                75      4   
Nat     5                     12                2.0                75      4   
        31                    12                2.0                75      4   
Nick    16                    12                2.0                32      4   
        42                    12                2.0                32      4   
Sachi   9                      8                2.0                75      4   
        29                     8                2.0                75      4   
Shawn   15                    12                3.0                60      4   
        35                    12                3.0                60      4   
Yelena  4                     16                0.5                75      4   
        27                    16                0.5                75      4   

                  cost_of_conducted_workouts  written_workouts_cost  \
name    block_id                                                      
Abelev  14                             600.0                    600   
        32                             600.0                    900   
Alan    19                             600.0                    600   
        39                             600.0                    600   
Anya    6                              480.0                    360   
        30                             480.0                    480   
Arne    18                             480.0                    480   
        41                             480.0                    480   
Daniel  13                             240.0                    360   
        33                             240.0                    480   
Dassi   8                              600.0                    600   
        40                             600.0                    600   
Dimitri 10                             480.0                    480   
        34                             480.0                    480   
D

### Analysis shows clients with largest ratio of unpaid workouts, where a positive number indicates a client who has paid more than they have used, and a Negative indicates a client that has recieved more workouts than they have paid for.

In [28]:
unpaid_service_ratio=result['unpaid_service_ratio'].sort_values(ascending=True)
unpaid_service_ratio

name     block_id
Yelena   27         -0.875000
         4          -0.875000
Misha    1          -0.500000
         36         -0.500000
Daniel   33         -0.500000
Nick     42         -0.333333
         16         -0.333333
Daniel   13         -0.333333
Mary     43         -0.333333
Nat      31         -0.333333
         5          -0.333333
Abelev   32         -0.333333
Mary     17         -0.333333
Sachi    9           0.000000
Shawn    15          0.000000
         35          0.000000
Masha    28          0.000000
         2           0.000000
Sachi    29          0.000000
Abelev   14          0.000000
Dino     37          0.000000
         21          0.000000
Dimitri  34          0.000000
         10          0.000000
Dassi    40          0.000000
         8           0.000000
Arne     41          0.000000
         18          0.000000
Anya     30          0.000000
Alan     39          0.000000
         19          0.000000
Igor     46          0.000000
         7           0

## Part 2: Determine Users with most/least skipped sessions

In [29]:
db_prescriptions

,name,workout_number,block_id,workouts_per_week,exercise_name,sets,reps,weight
0,Abelev,0,14,2,Side Planks,3,30,1
1,Abelev,0,14,2,Burpees,3,10,1
2,Abelev,0,14,2,Banded Walks,3,60,1
3,Abelev,0,14,2,Split Squat Jumps,3,10,1
4,Abelev,0,14,2,Banded Squats,3,10,1
...,...,...,...,...,...,...,...,...
2659,Yelena,15,27,4,Mountain Climbers,3,30,1
2660,Yelena,15,27,4,Back Squats,4,5,50
2661,Yelena,15,27,4,RFE Lunges,3,8,1
2662,Yelena,15,27,4,DB OH Press,4,5,10


In [30]:
conn=psycopg2.connect(dbname='cabral_fitness',
user='postgres',
password="Maninthemirror1995",
host='34.102.107.137',
port='5432',
sslmode='disable')

cursor=conn.cursor()
                    

table_name = 'actual_to_prescription'  # Replace with your table name

query = f"""
SELECT column_name 
FROM information_schema.columns 
WHERE table_name = '{table_name}'
"""

cursor.execute(query)
column_names = [row[0] for row in cursor.fetchall()]


In [31]:
conn.close()
column_names

['block_id', 'workout_number', 'session_id', 'id']

In [32]:
db_actual_to_prescription=pd.DataFrame(db_actual_to_prescription, columns=column_names)
db_actual_to_prescription

,block_id,workout_number,session_id,id
0,4,0,36,1
1,4,1,37,2
2,5,0,38,3
3,5,1,39,4
4,1,1,41,5
...,...,...,...,...
239,40,3,302,254
240,29,6,304,255
241,29,7,305,256
242,27,14,306,257


In [33]:
db_actual_to_prescription=db_actual_to_prescription.sort_values(by=['block_id', 'workout_number'], ascending=True)
db_actual_to_prescription=db_actual_to_prescription.drop(columns=['session_id','id'])
db_actual_to_prescription

,block_id,workout_number
5,1,0
4,1,1
15,1,2
16,1,3
24,1,4
...,...,...
219,42,3
225,42,4
208,43,1
220,46,0


In [34]:
original=db_prescriptions.drop(columns=['exercise_name','sets','reps','weight'])
original

,name,workout_number,block_id,workouts_per_week
0,Abelev,0,14,2
1,Abelev,0,14,2
2,Abelev,0,14,2
3,Abelev,0,14,2
4,Abelev,0,14,2
...,...,...,...,...
2659,Yelena,15,27,4
2660,Yelena,15,27,4
2661,Yelena,15,27,4
2662,Yelena,15,27,4


In [35]:
skipped_result=pd.merge(original, db_actual_to_prescription, how='inner', on=['block_id','workout_number'])
skipped_result

,name,workout_number,block_id,workouts_per_week
0,Abelev,0,14,2
1,Abelev,0,14,2
2,Abelev,0,14,2
3,Abelev,0,14,2
4,Abelev,0,14,2
...,...,...,...,...
1714,Yelena,15,27,4
1715,Yelena,15,27,4
1716,Yelena,15,27,4
1717,Yelena,15,27,4


In [36]:
# Group by 'name' and 'block_id' and get the unique 'workout_number' values for each group
original_grouped = original.groupby(['name', 'block_id'])['workout_number'].unique()
original_grouped

name     block_id
Abelev   14                                   [0, 1, 2, 3, 4, 5, 6, 7]
         32                     [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Alan     19                                   [0, 1, 2, 3, 4, 5, 6, 7]
         39                                   [0, 1, 2, 3, 4, 5, 6, 7]
Anya     6                                          [0, 1, 2, 3, 4, 5]
         30                                   [0, 1, 2, 3, 4, 5, 6, 7]
Arne     18                                   [0, 1, 2, 3, 4, 5, 6, 7]
         41                                   [0, 1, 2, 3, 4, 5, 6, 7]
Daniel   13                                         [0, 1, 2, 3, 4, 5]
         33                                   [0, 1, 2, 3, 4, 5, 6, 7]
Dassi    8                                    [0, 1, 2, 3, 4, 5, 6, 7]
         40                                   [0, 1, 2, 3, 4, 5, 6, 7]
Dimitri  10                                   [0, 1, 2, 3, 4, 5, 6, 7]
         34                                   [0, 1, 2, 3, 

In [37]:
skipped_grouped = skipped_result.groupby(['name', 'block_id'])['workout_number'].unique()
skipped_grouped


name     block_id
Abelev   14                               [0, 1, 2, 3, 4, 5, 6, 7]
         32                               [0, 1, 2, 3, 5, 6, 8, 9]
Alan     19                                  [0, 2, 3, 4, 5, 6, 7]
         39                                                 [1, 2]
Anya     6                                            [0, 3, 4, 5]
         30                                     [0, 1, 2, 3, 4, 5]
Arne     18                                  [0, 1, 2, 3, 4, 5, 6]
         41                                           [0, 1, 2, 3]
Daniel   13                                        [0, 2, 3, 4, 5]
         33                                                 [0, 2]
Dassi    8                                      [0, 1, 2, 3, 4, 5]
         40                                           [0, 1, 2, 3]
Dimitri  10                                        [0, 2, 3, 4, 5]
         34                                        [0, 1, 2, 3, 4]
Dino     21                                 

In [38]:
skipped_grouped.get

<bound method NDFrame.get of name     block_id
Abelev   14                               [0, 1, 2, 3, 4, 5, 6, 7]
         32                               [0, 1, 2, 3, 5, 6, 8, 9]
Alan     19                                  [0, 2, 3, 4, 5, 6, 7]
         39                                                 [1, 2]
Anya     6                                            [0, 3, 4, 5]
         30                                     [0, 1, 2, 3, 4, 5]
Arne     18                                  [0, 1, 2, 3, 4, 5, 6]
         41                                           [0, 1, 2, 3]
Daniel   13                                        [0, 2, 3, 4, 5]
         33                                                 [0, 2]
Dassi    8                                      [0, 1, 2, 3, 4, 5]
         40                                           [0, 1, 2, 3]
Dimitri  10                                        [0, 2, 3, 4, 5]
         34                                        [0, 1, 2, 3, 4]
Dino     21    

In [39]:
for (name, block_id), original_workouts in original_grouped.items():
    skipped_workouts = skipped_grouped.get((name, block_id), [])

    missing_workouts = set(original_workouts) - set(skipped_workouts)

    if missing_workouts:
        print(f"""For name={name} and block_id={block_id}, there are {len(missing_workouts)} missing workouts. 
              The missing workouts are {list(missing_workouts)}""")


For name=Abelev and block_id=32, there are 4 missing workouts. 
              The missing workouts are [10, 11, 4, 7]
For name=Alan and block_id=19, there are 1 missing workouts. 
              The missing workouts are [1]
For name=Alan and block_id=39, there are 6 missing workouts. 
              The missing workouts are [0, 3, 4, 5, 6, 7]
For name=Anya and block_id=6, there are 2 missing workouts. 
              The missing workouts are [1, 2]
For name=Anya and block_id=30, there are 2 missing workouts. 
              The missing workouts are [6, 7]
For name=Arne and block_id=18, there are 1 missing workouts. 
              The missing workouts are [7]
For name=Arne and block_id=41, there are 4 missing workouts. 
              The missing workouts are [4, 5, 6, 7]
For name=Daniel and block_id=13, there are 1 missing workouts. 
              The missing workouts are [1]
For name=Daniel and block_id=33, there are 6 missing workouts. 
              The missing workouts are [1, 3, 4, 5, 

In [40]:
with open('missing_workouts.txt', 'w') as file:  # 'w' mode will overwrite the existing file, use 'a' to append
    for (name, block_id), original_workouts in original_grouped.items():
        skipped_workouts = skipped_grouped.get((name, block_id), [])
        missing_workouts = set(original_workouts) - set(skipped_workouts)
        
        # if missing_workouts:
        file.write(f"""For name={name} and block_id={block_id}, there are {len(missing_workouts)} missing workouts.
              The missing workouts are {list(missing_workouts)}\n""")


In [41]:
from collections import defaultdict

In [42]:
missed=defaultdict(dict)
blocks=2
for (name, block_id), original_workouts in original_grouped.items():
    skipped_workouts = skipped_grouped.get((name, block_id), [])

    missing_workouts = set(original_workouts) - set(skipped_workouts)
    missed[name][block_id]=len(missing_workouts)

In [43]:
pd.Series(missed)

Abelev      {14: 0, 32: 4}
Alan        {19: 1, 39: 6}
Anya         {6: 2, 30: 2}
Arne        {18: 1, 41: 4}
Daniel      {13: 1, 33: 6}
Dassi        {8: 2, 40: 4}
Dimitri     {10: 3, 34: 3}
Dino        {21: 2, 37: 6}
Igor         {7: 2, 46: 6}
Mary       {17: 6, 43: 11}
Masha        {2: 1, 28: 0}
Misha        {1: 4, 36: 9}
Nat          {5: 2, 31: 7}
Nick        {16: 4, 42: 8}
Sachi        {9: 1, 29: 0}
Shawn       {15: 1, 35: 6}
Yelena       {4: 4, 27: 3}
dtype: object

### Part 3: Calculate an aggregate score based on the two metrics above

In [44]:
#Adjusted Anya's to zero to keep standard due to glitch in Database. Highest score is zero meaning I write no free workouts
#and the only workouts performed are ones we do together.

unpaid_service_ratio


name     block_id
Yelena   27         -0.875000
         4          -0.875000
Misha    1          -0.500000
         36         -0.500000
Daniel   33         -0.500000
Nick     42         -0.333333
         16         -0.333333
Daniel   13         -0.333333
Mary     43         -0.333333
Nat      31         -0.333333
         5          -0.333333
Abelev   32         -0.333333
Mary     17         -0.333333
Sachi    9           0.000000
Shawn    15          0.000000
         35          0.000000
Masha    28          0.000000
         2           0.000000
Sachi    29          0.000000
Abelev   14          0.000000
Dino     37          0.000000
         21          0.000000
Dimitri  34          0.000000
         10          0.000000
Dassi    40          0.000000
         8           0.000000
Arne     41          0.000000
         18          0.000000
Anya     30          0.000000
Alan     39          0.000000
         19          0.000000
Igor     46          0.000000
         7           0

In [45]:
dict_result = {}

for (name, number), value in unpaid_service_ratio.items():
    if name not in dict_result:
        dict_result[name] = {}
    dict_result[name][number] = value

dict_result

{'Yelena': {27: -0.875, 4: -0.875},
 'Misha': {1: -0.5, 36: -0.5},
 'Daniel': {33: -0.5, 13: -0.3333333333333333},
 'Nick': {42: -0.3333333333333333, 16: -0.3333333333333333},
 'Mary': {43: -0.3333333333333333, 17: -0.3333333333333333},
 'Nat': {31: -0.3333333333333333, 5: -0.3333333333333333},
 'Abelev': {32: -0.3333333333333333, 14: 0.0},
 'Sachi': {9: 0.0, 29: 0.0},
 'Shawn': {15: 0.0, 35: 0.0},
 'Masha': {28: 0.0, 2: 0.0},
 'Dino': {37: 0.0, 21: 0.0},
 'Dimitri': {34: 0.0, 10: 0.0},
 'Dassi': {40: 0.0, 8: 0.0},
 'Arne': {41: 0.0, 18: 0.0},
 'Anya': {30: 0.0, 6: 0.3333333333333333},
 'Alan': {39: 0.0, 19: 0.0},
 'Igor': {46: 0.0, 7: 0.0}}

In [46]:
dict_result['Nat']

{31: -0.3333333333333333, 5: -0.3333333333333333}

In [47]:
missed['Nat']

{5: 2, 31: 7}

In [48]:
adjusted_ratio_per_block = dict_result
total_missed_per_block = missed


min_missed = min(missed for blocks in total_missed_per_block.values() for missed in blocks.values())
max_missed = max(missed for blocks in total_missed_per_block.values() for missed in blocks.values())

min_ratio = min(ratio for blocks in adjusted_ratio_per_block.values() for ratio in blocks.values())
max_ratio = max(ratio for blocks in adjusted_ratio_per_block.values() for ratio in blocks.values())


### Aggregate Score Calculation

1. The aggregate score is calculated in a way where we can compare the 'value' of each client to the other based on how many free workouts they receieve (unpaid_workout_ratio) and how many workouts they ultimately skip (with me or on their own).

2. The score uses a simple formula where we deduct points from both sessions missed and from free workouts that are given. Free workouts are penalized at a higher rate since that consitutues labor that is not compensated for vs missed sessions consitutes compensation that is lost now (cancellations) but may be compensated later with rescheudling or just moving the workouts back a week. 
    * Note: This is by no means a perfect system since some cancellations are compensated for when cancelled within a certain timeframe and in those cases the workouts roll over and compensation would be collected. This case is not considered in the analysis.
    * In some cases programs roll over if a client misses two weeks for example, in these cases the analysis would show that they missed no workouts in the block if we decided to continue on the same program. How to determine the reduction for this is not considered in the analysis as all workouts would be compensated for but income for two weeks would be lost

3. Clients are scored on a scale from 0-10 where a 10 is a perfect score meaning every workout I write is compensated for and every workout is completed within the time is is intended to be completed

In [49]:
adjusted_ratio = dict_result
total_missed = missed

In [73]:
aggregate_scores = {}
for person in total_missed.keys():
    aggregate_scores[person] = {}
    for block in total_missed[person].keys():
        missed_deduction = total_missed[person][block] * 0.5 #Deduct 50% of score for each missed workout
        ratio_deduction = abs(adjusted_ratio[person][block]) * 1 #Deduct 100% of unpaid service ratio
        aggregate_scores[person][block] = 10 - missed_deduction - ratio_deduction


In [74]:
aggregate_scores

{'Abelev': {14: 10.0, 32: 7.666666666666667},
 'Alan': {19: 9.5, 39: 7.0},
 'Anya': {6: 8.666666666666666, 30: 9.0},
 'Arne': {18: 9.5, 41: 8.0},
 'Daniel': {13: 9.166666666666666, 33: 6.5},
 'Dassi': {8: 9.0, 40: 8.0},
 'Dimitri': {10: 8.5, 34: 8.5},
 'Dino': {21: 9.0, 37: 7.0},
 'Igor': {7: 9.0, 46: 7.0},
 'Mary': {17: 6.666666666666667, 43: 4.166666666666667},
 'Masha': {2: 9.5, 28: 10.0},
 'Misha': {1: 7.5, 36: 5.0},
 'Nat': {5: 8.666666666666666, 31: 6.166666666666667},
 'Nick': {16: 7.666666666666667, 42: 5.666666666666667},
 'Sachi': {9: 9.5, 29: 10.0},
 'Shawn': {15: 9.5, 35: 7.0},
 'Yelena': {4: 7.125, 27: 7.625}}

In [84]:
#New loop to replace unique block ids with block 1/2 etc

new_aggregate_scores = {}
for person, blocks in aggregate_scores.items():
    new_blocks = {}
    for index, (block_id, score) in enumerate(sorted(blocks.items()), start=1):
        new_blocks['Block '+str(index)] = score
    new_aggregate_scores[person] = new_blocks




In [85]:
new_aggregate_scores

{'Abelev': {'Block 1': 10.0, 'Block 2': 7.666666666666667},
 'Alan': {'Block 1': 9.5, 'Block 2': 7.0},
 'Anya': {'Block 1': 8.666666666666666, 'Block 2': 9.0},
 'Arne': {'Block 1': 9.5, 'Block 2': 8.0},
 'Daniel': {'Block 1': 9.166666666666666, 'Block 2': 6.5},
 'Dassi': {'Block 1': 9.0, 'Block 2': 8.0},
 'Dimitri': {'Block 1': 8.5, 'Block 2': 8.5},
 'Dino': {'Block 1': 9.0, 'Block 2': 7.0},
 'Igor': {'Block 1': 9.0, 'Block 2': 7.0},
 'Mary': {'Block 1': 6.666666666666667, 'Block 2': 4.166666666666667},
 'Masha': {'Block 1': 9.5, 'Block 2': 10.0},
 'Misha': {'Block 1': 7.5, 'Block 2': 5.0},
 'Nat': {'Block 1': 8.666666666666666, 'Block 2': 6.166666666666667},
 'Nick': {'Block 1': 7.666666666666667, 'Block 2': 5.666666666666667},
 'Sachi': {'Block 1': 9.5, 'Block 2': 10.0},
 'Shawn': {'Block 1': 9.5, 'Block 2': 7.0},
 'Yelena': {'Block 1': 7.125, 'Block 2': 7.625}}

In [86]:
# First find the unique block names
block_names = set()
for scores in new_aggregate_scores.values():
    block_names.update(scores.keys())

# Then for each block name create a separate dictionary and sort it
sorted_scores_blocks = {}
for block_name in block_names:
    block_scores = {person: scores[block_name] for person, scores in new_aggregate_scores.items() if block_name in scores}
    sorted_scores_blocks[block_name] = sorted(block_scores.items(), key=lambda item: item[1], reverse=True)


In [87]:
sorted_scores_blocks

{'Block 2': [('Masha', 10.0),
  ('Sachi', 10.0),
  ('Anya', 9.0),
  ('Dimitri', 8.5),
  ('Arne', 8.0),
  ('Dassi', 8.0),
  ('Abelev', 7.666666666666667),
  ('Yelena', 7.625),
  ('Alan', 7.0),
  ('Dino', 7.0),
  ('Igor', 7.0),
  ('Shawn', 7.0),
  ('Daniel', 6.5),
  ('Nat', 6.166666666666667),
  ('Nick', 5.666666666666667),
  ('Misha', 5.0),
  ('Mary', 4.166666666666667)],
 'Block 1': [('Abelev', 10.0),
  ('Alan', 9.5),
  ('Arne', 9.5),
  ('Masha', 9.5),
  ('Sachi', 9.5),
  ('Shawn', 9.5),
  ('Daniel', 9.166666666666666),
  ('Dassi', 9.0),
  ('Dino', 9.0),
  ('Igor', 9.0),
  ('Anya', 8.666666666666666),
  ('Nat', 8.666666666666666),
  ('Dimitri', 8.5),
  ('Nick', 7.666666666666667),
  ('Misha', 7.5),
  ('Yelena', 7.125),
  ('Mary', 6.666666666666667)]}

### Step 4: Export Data for Tableau Visulaization

In [91]:
with open('missing_workouts.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Name", "Block ID", "Number of Missing Workouts", "Missing Workouts"])  # write header
    for (name, block_id), original_workouts in original_grouped.items():
        skipped_workouts = skipped_grouped.get((name, block_id), [])
        missing_workouts = set(original_workouts) - set(skipped_workouts)
        writer.writerow([name, block_id, len(missing_workouts), list(missing_workouts)])


In [90]:
#Two different approaches to visualizing this data, this one may be more grnaular given information
#aboutb the workouts missed

with open('missing_workouts.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Name", "Block ID", "Missing Workout"])  # write header
    for (name, block_id), original_workouts in original_grouped.items():
        skipped_workouts = skipped_grouped.get((name, block_id), [])
        missing_workouts = set(original_workouts) - set(skipped_workouts)
        for workout in missing_workouts:
            writer.writerow([name, block_id, workout])


In [96]:
new_missed = {}
for person, blocks in missed.items():
    new_blocks = {}
    for index, (block_id, scbore) in enumerate(sorted(blocks.items()), start=1):
        new_blocks['Block '+str(index)] = score
    new_missed[person] = new_blocks

In [136]:
new_missed

{'Abelev': {'Block 1': 0, 'Block 2': 4},
 'Alan': {'Block 1': 1, 'Block 2': 6},
 'Anya': {'Block 1': 2, 'Block 2': 2},
 'Arne': {'Block 1': 1, 'Block 2': 4},
 'Daniel': {'Block 1': 1, 'Block 2': 6},
 'Dassi': {'Block 1': 2, 'Block 2': 4},
 'Dimitri': {'Block 1': 3, 'Block 2': 3},
 'Dino': {'Block 1': 2, 'Block 2': 6},
 'Igor': {'Block 1': 2, 'Block 2': 6},
 'Mary': {'Block 1': 6, 'Block 2': 11},
 'Masha': {'Block 1': 1, 'Block 2': 0},
 'Misha': {'Block 1': 4, 'Block 2': 9},
 'Nat': {'Block 1': 2, 'Block 2': 7},
 'Nick': {'Block 1': 4, 'Block 2': 8},
 'Sachi': {'Block 1': 1, 'Block 2': 0},
 'Shawn': {'Block 1': 1, 'Block 2': 6},
 'Yelena': {'Block 1': 4, 'Block 2': 3}}

In [99]:
new_service_ratio = {}
for person, blocks in unpaid_csv.items():
    new_blocks = {}
    for index, (block_id, score) in enumerate(sorted(blocks.items()), start=1):
        new_blocks['Block '+str(index)] = score
    new_service_ratio[person] = new_blocks

new_service_ratio


{'Yelena': {'Block 1': -0.875, 'Block 2': -0.875},
 'Misha': {'Block 1': -0.5, 'Block 2': -0.5},
 'Daniel': {'Block 1': -0.3333333333333333, 'Block 2': -0.5},
 'Nick': {'Block 1': -0.3333333333333333, 'Block 2': -0.3333333333333333},
 'Mary': {'Block 1': -0.3333333333333333, 'Block 2': -0.3333333333333333},
 'Nat': {'Block 1': -0.3333333333333333, 'Block 2': -0.3333333333333333},
 'Abelev': {'Block 1': 0.0, 'Block 2': -0.3333333333333333},
 'Sachi': {'Block 1': 0.0, 'Block 2': 0.0},
 'Shawn': {'Block 1': 0.0, 'Block 2': 0.0},
 'Masha': {'Block 1': 0.0, 'Block 2': 0.0},
 'Dino': {'Block 1': 0.0, 'Block 2': 0.0},
 'Dimitri': {'Block 1': 0.0, 'Block 2': 0.0},
 'Dassi': {'Block 1': 0.0, 'Block 2': 0.0},
 'Arne': {'Block 1': 0.0, 'Block 2': 0.0},
 'Anya': {'Block 1': 0.3333333333333333, 'Block 2': 0.0},
 'Alan': {'Block 1': 0.0, 'Block 2': 0.0},
 'Igor': {'Block 1': 0.0, 'Block 2': 0.0}}

In [101]:
pd.Series(new_service_ratio).to_csv('unpaid_service_ratio.csv')

Yelena                {'Block 1': -0.875, 'Block 2': -0.875}
Misha                     {'Block 1': -0.5, 'Block 2': -0.5}
Daniel     {'Block 1': -0.3333333333333333, 'Block 2': -0.5}
Nick       {'Block 1': -0.3333333333333333, 'Block 2': -0...
Mary       {'Block 1': -0.3333333333333333, 'Block 2': -0...
Nat        {'Block 1': -0.3333333333333333, 'Block 2': -0...
Abelev      {'Block 1': 0.0, 'Block 2': -0.3333333333333333}
Sachi                       {'Block 1': 0.0, 'Block 2': 0.0}
Shawn                       {'Block 1': 0.0, 'Block 2': 0.0}
Masha                       {'Block 1': 0.0, 'Block 2': 0.0}
Dino                        {'Block 1': 0.0, 'Block 2': 0.0}
Dimitri                     {'Block 1': 0.0, 'Block 2': 0.0}
Dassi                       {'Block 1': 0.0, 'Block 2': 0.0}
Arne                        {'Block 1': 0.0, 'Block 2': 0.0}
Anya         {'Block 1': 0.3333333333333333, 'Block 2': 0.0}
Alan                        {'Block 1': 0.0, 'Block 2': 0.0}
Igor                    

In [102]:
#Select Dictionaries to export and use pandas.to_csv to export to csv file

new_service_ratio
pd.Series(new_service_ratio).to_csv('unpaid_service_ratio.csv')

new_missed
pd.Series(new_missed).to_csv('total_missed.csv')

sorted_scores_blocks
pd.Series(sorted_scores_blocks).to_csv('aggregate_scores.csv')

new_aggregate_scores
pd.Series(new_aggregate_scores).to_csv('new_aggregate_scores.csv')

### Modifying how some of the data looks to make it more appropriate for Tableau

In [118]:
sorted_scores_2=pd.DataFrame(sorted_scores_blocks)['Block 2']
sorted_scores_1=pd.DataFrame(sorted_scores_blocks)['Block 1']

names = [name for name, score in sorted_scores_1]
scores = [score for name, score in sorted_scores_1]

score_series_1 = pd.Series(scores, index=names)

# Print the series
print(score_series_1)
score_series_1.to_csv('block_1_scores.csv')


Abelev     10.000000
Alan        9.500000
Arne        9.500000
Masha       9.500000
Sachi       9.500000
Shawn       9.500000
Daniel      9.166667
Dassi       9.000000
Dino        9.000000
Igor        9.000000
Anya        8.666667
Nat         8.666667
Dimitri     8.500000
Nick        7.666667
Misha       7.500000
Yelena      7.125000
Mary        6.666667
dtype: float64


In [119]:
names = [name for name, score in sorted_scores_2]
scores = [score for name, score in sorted_scores_2]

score_series_2 = pd.Series(scores, index=names)
score_series_2
score_series_2.to_csv('block_2_scores.csv')

In [129]:
unpaid_services_block_1=pd.DataFrame(new_service_ratio).T
unpaid_services_block_1


,Block 1,Block 2
Yelena,-0.875000,-0.875000
Misha,-0.500000,-0.500000
Daniel,-0.333333,-0.500000
Nick,-0.333333,-0.333333
Mary,-0.333333,-0.333333
Nat,-0.333333,-0.333333
Abelev,0.000000,-0.333333
Sachi,0.000000,0.000000
Shawn,0.000000,0.000000
Masha,0.000000,0.000000


In [132]:
unpaid_services_block_1.to_csv('unpaid_service_ratio.csv')

When presenting this in Tableau you can see that the number is kind of hard to understand. what is a negative .875? 

We are going to convert it into a positive percentage so thhat the context is easier to understand. 87.5% of your workouts are unpaid.

In [134]:
unpaid_services_block_1=unpaid_services_block_1.abs()*100
unpaid_services_block_1

,Block 1,Block 2
Yelena,87.500000,87.500000
Misha,50.000000,50.000000
Daniel,33.333333,50.000000
Nick,33.333333,33.333333
Mary,33.333333,33.333333
Nat,33.333333,33.333333
Abelev,0.000000,33.333333
Sachi,0.000000,0.000000
Shawn,0.000000,0.000000
Masha,0.000000,0.000000


In [135]:
unpaid_services_block_1.to_csv('unpaid_service_ratio.csv')

### Modifying formatting for missing workouts to say Block 1/Block 2

In [139]:
missing_workouts=pd.DataFrame(new_missed).T
missing_workouts.to_csv('missing_workouts.csv')
missing_workouts


,Block 1,Block 2
Abelev,0,4
Alan,1,6
Anya,2,2
Arne,1,4
Daniel,1,6
Dassi,2,4
Dimitri,3,3
Dino,2,6
Igor,2,6
Mary,6,11


### Some Insights After Digging into the Data

1. I should date the blocks in the database and keep track of when they should be completed by. This will make analysis easier
2. I should also stay consistent on whether when workouts are missed we skip them or continue on, this isn't really an easy answer though and shouldn't be unilaterally applied to everyone. it is very much judged case by case
3. Integrating and storing my calendar information into my database and linking them with clients could be a cool step to make it easier to assess the true value of missed workouts even if we continue on the same block. In the analysis as it stands, a client could miss 3 weeks, train once, miss 3 more weeks, and then finish out a block, and if we continued on then there would be no drop in value and that is not accurate.